## Importação das bibliotecas

In [25]:
import numpy as np
import pandas as pd
import plotly
import plotly.offline as py
import yfinance as yf
import datetime
import plotly.graph_objs as go

## Sem esse comando os graficos não sao exibidos no notebook

In [26]:
from plotly.offline import plot, iplot
plotly.offline.init_notebook_mode(connected=True)

## Recebe os inputs do usuário e busca as cotações na base de dados do Yahoo

In [27]:
# Recebe o input do usuario com o nome da ação
stock = input('Digite o código da ação que deseja analisar')
stock = stock + '.SA'
# Recebe do usuário o valor das médias 
num_mm_short = int(input('Digite um valor númerico para a média móvel curta'))
num_mm_long = int(input('Digite um valor numerico para a média móvel longa'))
mean_short = 'MM' + str(num_mm_short)
mean_long = 'MM' + str(num_mm_long)

# Define o periodo de busca do historico de preços
ini_date = datetime.date(2020,1,1)
end_date = datetime.date(2020,12,31)

df_stock_prices = yf.download(stock, start=ini_date, end=end_date)
df_stock_prices.reset_index(drop=False, inplace=True)

df_stock_prices.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-01-02,30.510000,30.700001,30.309999,30.700001,30.698339,37774500
1,2020-01-03,30.879999,31.240000,30.450001,30.450001,30.448353,71595600
2,2020-01-06,30.430000,30.940001,29.950001,30.809999,30.808332,81844000
3,2020-01-07,30.820000,30.879999,30.469999,30.690001,30.688339,32822000
4,2020-01-08,30.690001,30.770000,30.240000,30.500000,30.498348,48215600


## Cria as colunas de média móvel no Data Frame e preenche com zero os campos com 'NaN'

In [29]:
df_stock_prices[mean_short] = df_stock_prices.Close.rolling(window=num_mm_short).mean()
df_stock_prices[mean_long] = df_stock_prices.Close.rolling(window=num_mm_long).mean()
df_stock_prices['SPREAD_MM'] = df_stock_prices[mean_short] - df_stock_prices[mean_long]

df_stock_prices[mean_short] = df_stock_prices[mean_short].fillna(0)
df_stock_prices[mean_long] = df_stock_prices[mean_long].fillna(0)
df_stock_prices.head(25)

,Date,Open,High,Low,Close,Adj Close,Volume,MM3,MM5
0,2020-01-02,30.510000,30.700001,30.309999,30.700001,30.698339,37774500,0.000000,0.000000
1,2020-01-03,30.879999,31.240000,30.450001,30.450001,30.448353,71595600,0.000000,0.000000
2,2020-01-06,30.430000,30.940001,29.950001,30.809999,30.808332,81844000,30.653334,0.000000
3,2020-01-07,30.820000,30.879999,30.469999,30.690001,30.688339,32822000,30.650000,0.000000
4,2020-01-08,30.690001,30.770000,30.240000,30.500000,30.498348,48215600,30.666667,30.630000
5,2020-01-09,30.469999,30.620001,30.250000,30.400000,30.398355,36102700,30.530000,30.570000
6,2020-01-10,30.389999,30.440001,30.129999,30.270000,30.268362,25397500,30.390000,30.534000
7,2020-01-13,30.350000,30.430000,30.209999,30.330000,30.328358,30676800,30.333333,30.438000
8,2020-01-14,30.230000,30.270000,29.770000,30.000000,29.998377,39796500,30.200000,30.300000
9,2020-01-15,29.900000,29.930000,29.540001,29.549999,29.548399,34405700,29.960000,30.110000


## Plota o gráfico com o preço de fechamento e as médias móveis

In [44]:
close_prices = go.Scatter(
                            x = df_stock_prices['Date'],
                            y = df_stock_prices['Close'],
                            name = stock + 'Preço de Fechamento',
                            line = dict(color = '#330000'),
                            opacity = 1.0)

mm_s = go.Scatter(
                    x = df_stock_prices['Date'],
                    y = df_stock_prices[mean_short],
                    name = 'Média Móvel Curta',
                    line = dict(color = '#B2FF66'),
                    opacity = 1.0)

mm_l = go.Scatter(
                    x = df_stock_prices['Date'],
                    y = df_stock_prices[mean_long],
                    name = 'Média Móvel Longa',
                    line = dict(color = '#FF00FF'),
                    opacity = 1.0)

layout = go.Layout(
                    autosize=False,
                    legend_orientation='h',
                    width=1000,
                    height=400)

data_prices = [close_prices,mm_s,mm_l]

fig = go.Figure(data=data_prices, layout=layout)

py.iplot(fig)

## Faz o backtest da estratégia 